In [1]:
import numpy as np
from tic_env import TictactoeEnv, OptimalPlayer
from utils import play_game, Metric
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm_notebook
import itertools
import torch
import torch.nn as nn

In [72]:
def Metric_Q(policy,Q_table,optimal=False):
    N_wins=0
    N_losses=0
    N=0
    Turns = np.array([['X','O']]*250+[['O','X']]*250)
    for i in range(500):
        np.random.seed()

        
        if optimal: 
            player_test = OptimalPlayer(epsilon=0., player=Turns[i,1])
        if not optimal:
            player_test = OptimalPlayer(epsilon=1., player=Turns[i,1])

        player_new = policy(player=Turns[i,0],epsilon=0)
        env=TictactoeEnv()
        while not env.end:
            if env.current_player == player_new.player:
                state=get_state(env.grid,player_new)
                move = player_new.act(state,env.grid,Q_table)       
            else:
                move = player_test.act(env.grid)

            if not isinstance(move,tuple): 
                    move=(int(move/3),move%3)
            env.step(move, print_grid=False)
                
        if env.winner==player_new.player:
            N_wins+=1
        if env.winner==player_test.player:
            N_losses+=1
        N+=1
        env.reset()               
    return (N_wins - N_losses)/N

class deep_Q_player:
    def __init__(self,player='X', epsilon=0):
        self.player=player
        self.epsilon=epsilon
    
        
    def act(self,state,model):
        
        action_scores = model(state)
        
        action = np.random.choice(np.where(action_scores == np.max(action_scores))[0])
        if np.random.random() < self.epsilon:
            action = np.random.choice(range(9))
        
        return action

def play_game_self_deepQ(env,model,p1,p2):
    R_t=[]
    while not env.end:
        if env.current_player == p1.player:
            state1=grid2tensor(env.grid,p1.player)
            action1=p1.act(state1,model)
            if env.check_valid():
                env.step((int(action1/3), action1 % 3), print_grid=False)
                reward1=env.reward(p1.player)
                new_state1=grid2tensor(env.grid,p1.player)
                R_t.append(state1,action1,reward1,new_state1)
            else:
                reward1=-1
                R_t.append(state1,action1,reward1,None)
                break

        if env.current_player == p2.player:
            state2=grid2tensor(env.grid,p2.player)
            action2=p2.act(state2,model)
            if env.check_valid():
                env.step((int(action2/3), action2 % 3), print_grid=False)
                reward2=env.reward(p2.player)
                new_state2=grid2tensor(env.grid,p2.player)
                R_t.append(state2,action2,reward2,new_state2)
            else:
                reward2=-1
                R_t.append(state2,action2,reward2,None)
                break
        
    return env, R_t

def Q_loss(r,max_new_Q_val,Q_val,gamma=.99):
        return .5(r + gamma*max_new_Q_val - Q_val).pow(2)

In [7]:
gamma=.99
buff_size=10000
batch_size=64
lr=5e-4

class DQN(nn.Module):
    def __init__(self):
        super(DQN,self).__init__()

        self.fc=nn.Sequential(
            nn.Linear(3*3*2,128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,9)
        )
    
    def forward(self,x):
        x=x.view(18)
        x=self.fc(x)
        return x

        

In [63]:
def grid2tensor(grid,player):
    tens=torch.zeros(3,3,2)
    grid=torch.from_numpy(grid)
    if player=='X':
        player_index=torch.nonzero(grid==1,as_tuple=False)
        op_index=torch.nonzero(grid==-1,as_tuple=False)
    if player=='O':
        player_index=torch.nonzero(grid==-1,as_tuple=False)
        op_index=torch.nonzero(grid==1,as_tuple=False)
    tens[player_index[:,0],player_index[:,1],0]=1
    tens[op_index[:,0],op_index[:,1],1]=1
    return tens

In [65]:
model=DQN()
R=[]
target=model
p1=deep_Q_player('X')
p2=deep_Q_player('O')
#should we first play all games, then train? or do it at the same time?
for game in range(nb_games):
    env=TictactoeEnv()
    env, R_t=play_game_self_deepQ(env,model,p1,p2)
    R.extend(R_t)
    sample=sample_from(R)
    loss=Q_loss(rews,max_new_Q_vals,Q_vals)
    if game%target_update_step==0:
        target=model
    



tensor([[[0., 0.],
         [0., 1.],
         [0., 0.]],

        [[0., 0.],
         [1., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.]]])

In [69]:
def m(a):
    a=0
a=1
print(a)
m(a)
print(a)

1
1
